In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder
import os

2026-01-18 13:17:59.567460: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-18 13:17:59.617868: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-18 13:18:01.059398: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
MODEL_DIR = "../models"

print("Sedang load model...")
lstm_model   = load_model(os.path.join(MODEL_DIR, "lstm_model.keras"))
bilstm_model = load_model(os.path.join(MODEL_DIR, "bilstm_model.keras"))
print("Model berhasil di-load!")

Sedang load model...


I0000 00:00:1768717082.388193   64111 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4132 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model berhasil di-load!


In [ ]:
def get_flops(model):
    full_model = tf.function(lambda x: model(x))
    
    input_specs = []
    for inp in model.inputs:
        
        dims = [dim for dim in inp.shape[1:]] 
        shape = [1] + dims
        
        input_specs.append(tf.TensorSpec(shape, inp.dtype))
        
    concrete_func = full_model.get_concrete_function(input_specs)

    frozen_func = concrete_func

    run_meta = tf.compat.v1.RunMetadata()
    opts = ProfileOptionBuilder.float_operation()
    opts['output'] = 'none'
    
    flops = profile(graph=frozen_func.graph,
                    run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops

In [4]:
print("-" * 30)
print("HASIL PERHITUNGAN FLOPS (Efisiensi Komputasi)")
print("-" * 30)

# --- Cek LSTM ---
try:
    flops_lstm = get_flops(lstm_model)
    print(f"1. LSTM Model")
    print(f"   - Total Operasi : {flops_lstm:,} Ops")
    print(f"   - Mega FLOPs    : {flops_lstm / 10**6:.4f} MFLOPs")
except Exception as e:
    print(f"Gagal hitung LSTM: {e}")

print("\n")

# --- Cek BiLSTM ---
try:
    flops_bilstm = get_flops(bilstm_model)
    print(f"2. BiLSTM Model")
    print(f"   - Total Operasi : {flops_bilstm:,} Ops")
    print(f"   - Mega FLOPs    : {flops_bilstm / 10**6:.4f} MFLOPs")
except Exception as e:
    print(f"Gagal hitung BiLSTM: {e}")

print("-" * 30)

------------------------------
HASIL PERHITUNGAN FLOPS (Efisiensi Komputasi)
------------------------------
Instructions for updating:
This API was designed for TensorFlow v1. See https://www.tensorflow.org/guide/migrate for instructions on how to migrate your code to TensorFlow v2.
1. LSTM Model
   - Total Operasi : 163,585 Ops
   - Mega FLOPs    : 0.1636 MFLOPs


2. BiLSTM Model
   - Total Operasi : 165,633 Ops
   - Mega FLOPs    : 0.1656 MFLOPs
------------------------------
